
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

-----

    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

-----

In [19]:


# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
import pandas as pd
import numpy as np
data = pd.read_csv("data/Corona_NLP_train.csv", encoding="latin1")
#print(data.head())
#print(data.columns.values)
data = data[data["Sentiment"] != "Neutral"]
# The line below might be needed: It's the same filtering!
# data = data[data["Sentiment"].isin([
#     "Positive", "Extremely Positive", "Negative", "Extremely Negative"
# ])]


X = data["OriginalTweet"].to_numpy()
# print(f"X Shape is: {X.shape}")


Y = (
    (data["Sentiment"] == "Positive") |
    (data["Sentiment"] == "Extremely Positive")
).astype(int).to_numpy()


print(Y.shape)
# print(f"This is Y: {Y}")

n = len(Y)
n_train = int(0.6 * n)
n_test = int(0.15 * n)

X_train = X[:n_train]
X_test = X[n_train:n_train+n_test]
X_valid = X[n_train+n_test:]

Y_train = Y[:n_train]
Y_test = Y[n_train:n_train+n_test]
Y_valid = Y[n_train+n_test:]

(33444,)


-----
2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**

-----

In [20]:

# Part 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from scipy import optimize
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    precision_score,
    recall_score,
)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

model = Pipeline([
    ('CV', CountVectorizer()),
    ('lr', LogisticRegression(max_iter=1000))
])

model.fit(X_train, Y_train)

,steps,"[('CV', ...), ('lr', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


-----
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.

-----

In [21]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

y_pred = model.predict(X_test)
y_true = Y_test

precision_0 = precision_score(y_true, y_pred, pos_label=0)
precision_1 = precision_score(y_true, y_pred, pos_label=1)
recall_0 = recall_score(y_true, y_pred, pos_label=0)
recall_1 = recall_score(y_true, y_pred, pos_label=1)

print(f"This is the Recall 0: {recall_0} \nThis is Recall 1: {recall_1} \n \nThis is Precision 0: {precision_0} \nThis is Precision 1: {precision_1}")

This is the Recall 0: 0.8455531453362256 
This is Recall 1: 0.8801180376244928 
 
This is Precision 0: 0.8570800351802991 
This is Precision 1: 0.8701677607585704


-----
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 


-----

In [22]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    y_proba = model.predict_proba(X)[:, 1]  # = P(Y = 1 ∣ X)
    y_pred = (y_proba >= threshold).astype(int) 
    
    y_true = Y
    
    fn_cost = 1
    fp_cost = 5
    
    FN = np.sum((y_true == 1) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))

    return ((FN * fn_cost) + (FP * fp_cost)) / (len(y_pred))

-----
5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.

-----

In [26]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.
thresholds = np.arange(0, 1.01, 0.01)
best_t = 0
best_cost = np.inf

for t in thresholds:
    problem5_cost = cost(model, t, X_test, y_true)
    if (problem5_cost < best_cost):
        best_cost = problem5_cost
        best_t = t
    
optimal_threshold = best_t
cost_at_optimal_threshold = best_cost

print(f"Optimal Threshold: {optimal_threshold} \nCost at optimal {cost_at_optimal_threshold}")

Optimal Threshold: 0.81 
Cost at optimal 0.27352472089314195


-----
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.

-----

In [24]:

# Part 6

alpha = 0.01
a,b = 0,5
n = len(Y_valid)

epsilon = (b-a)*np.sqrt(np.log(alpha/2) / (-2*n))
epsilon1 = (b-a) * np.sqrt(np.log(2/alpha) / (2*n))


cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

lower = cost_at_optimal_threshold_valid - epsilon
upper = cost_at_optimal_threshold_valid + epsilon

lower = cost_at_optimal_threshold_valid - epsilon1
upper = cost_at_optimal_threshold_valid + epsilon1

cost_interval_valid = (lower, upper)

print(f"The cost interval is: {cost_interval_valid} using epsilon")
print(f"The cost interval is: {cost_interval_valid} using epsilon1")

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

The cost interval is: (np.float64(0.1870148471781446), np.float64(0.3650062004181242)) using epsilon
The cost interval is: (np.float64(0.1870148471781446), np.float64(0.3650062004181242)) using epsilon1


-----
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?
-----

In [25]:

# Part 7
import numpy as np

# Compute C on the validation set
probs_valid = model.predict_proba(X_valid)[:, 1]
I_valid = (probs_valid >= optimal_threshold).astype(int)

# C = (1 - 1_{f(X)>=t}) * Y + 5 * (1 - Y) * 1_{f(X)>=t}
C_valid = (1 - I_valid) * Y_valid + 5 * (1 - Y_valid) * I_valid

# Empirical variance of C (population variance, denominator n)
variance_of_C = C_valid.var()

# Bennett's inequality-based confidence interval (99%)
n = len(C_valid)
b = 5.0
delta = 0.01

def h(u):
    return (1.0 + u) * np.log(1.0 + u) - u

def bennett_epsilon(n, var_hat, b, delta):
    """
    Solve for epsilon in the two-sided Bennett inequality:
        2 * exp( - n * var_hat / b^2 * h(b * epsilon / var_hat) ) = delta
    using a simple binary search over epsilon in [0, b].
    """
    if var_hat == 0.0:
        # All costs are identical, no deviation
        return 0.0

    target = (b ** 2) * np.log(2.0 / delta) / (n * var_hat)

    lo, hi = 0.0, b
    for _ in range(100):
        mid = 0.5 * (lo + hi)
        u = b * mid / var_hat
        val = h(u)
        if val < target:
            lo = mid
        else:
            hi = mid
    return hi

epsilon_B = bennett_epsilon(n, variance_of_C, b, delta)

interval_of_C = (
    cost_at_optimal_threshold_valid - epsilon_B,
    cost_at_optimal_threshold_valid + epsilon_B
)

print("Empirical variance of C on validation set:", variance_of_C)
print("Bennett 99% confidence interval for E[C]:", interval_of_C)



assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Empirical variance of C on validation set: 0.639914818353732
Bennett 99% confidence interval for E[C]: (np.float64(0.24649627594655238), np.float64(0.3055247716497164))
